In [36]:
from pprint import pprint
import pandas as pd
import json

In [37]:
def json_to_dfs(dict_dados):
    dict_linhas = {}
    for tipo_colheita in dict_dados:
        colheita_categorias = dict_dados[tipo_colheita]
        dict_linhas[tipo_colheita] = []
        for categoria in colheita_categorias:
            categoria_itens = colheita_categorias[categoria]
            for nome_item in categoria_itens:
                item = categoria_itens[nome_item]
                item['tipo_colheita'] = tipo_colheita
                item['categoria'] = categoria
                
                dict_linhas[tipo_colheita].append(item)
    
    dict_dfs = {}
    for tipo_colheita in dict_linhas:
        dados_atual = dict_linhas[tipo_colheita]
        df_atual = pd.DataFrame(dados_atual)
        dict_dfs[tipo_colheita] = df_atual
    
    return dict_dfs

In [38]:
with open('dados_completos.json', 'r') as json_dados:
    dict_dados = json.loads(json_dados.read())

dict_dfs = json_to_dfs(dict_dados)
dict_dfs['Crops']

,url_img,custo,moeda_custo,estacoes,xp,ganho_base,moeda_ganho,tempo,tipo_recurso,nome,link,tipo_colheita,categoria
0,https://vignette.wikia.nocookie.net/farmtogeth...,35.0,dinheiro,"[fall, spring, winter]",2.0,60.0,dinheiro,20.0,Vegetables,Carrot,https://farmtogether.fandom.com/wiki/Carrot,Crops,Vegetables
1,https://vignette.wikia.nocookie.net/farmtogeth...,15.0,dinheiro,"[fall, spring, winter, summer]",1.0,30.0,dinheiro,10.0,Vegetables,Lettuce,https://farmtogether.fandom.com/wiki/Lettuce,Crops,Vegetables
2,https://vignette.wikia.nocookie.net/farmtogeth...,100.0,dinheiro,"[spring, summer]",5.0,140.0,dinheiro,45.0,Vegetables,Cabbage,https://farmtogether.fandom.com/wiki/Cabbage,Crops,Vegetables
3,https://vignette.wikia.nocookie.net/farmtogeth...,500.0,dinheiro,"[fall, spring]",65.0,850.0,dinheiro,1080.0,Vegetables,Lombardy Cabbage,https://farmtogether.fandom.com/wiki/Lombardy_...,Crops,Vegetables
4,https://vignette.wikia.nocookie.net/farmtogeth...,4200.0,dinheiro,"[fall, spring, winter]",10.0,4300.0,dinheiro,20.0,Vegetables,Purple Carrot,https://farmtogether.fandom.com/wiki/Purple_Ca...,Crops,Vegetables
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,https://vignette.wikia.nocookie.net/farmtogeth...,300.0,dinheiro,"[fall, spring, winter, summer]",2.0,1.0,diamante,20.0,No Resource,Toy Pinwheels,https://farmtogether.fandom.com/wiki/Toy_Pinwh...,Crops,Event
93,https://vignette.wikia.nocookie.net/farmtogeth...,800.0,dinheiro,"[fall, spring, winter, summer]",18.0,1.0,diamante,180.0,No Resource,Musical Root,https://farmtogether.fandom.com/wiki/Musical_Root,Crops,Event
94,https://vignette.wikia.nocookie.net/farmtogeth...,500.0,dinheiro,"[fall, spring, winter, summer]",11.0,1.0,diamante,90.0,No Resource,Magic Crystal,https://farmtogether.fandom.com/wiki/Magic_Cry...,Crops,Event
95,https://vignette.wikia.nocookie.net/farmtogeth...,1000.0,dinheiro,"[fall, spring, winter, summer]",22.0,1.0,diamante,240.0,No Resource,Party Candles,https://farmtogether.fandom.com/wiki/Party_Can...,Crops,Event


# Passar por cada df e incluir a coluna de ganho/minuto

## Padronização dos ganho em dinheiros, não em diamantes

### Dict de conversão

In [39]:
DICT_CONVERSAO = {
    'diamante_dinheiro': 1000,
    'fruit_dinheiro': 500,
    'vegetable_dinheiro': 250,
    'grape_dinheiro': 400,
    'flower_dinheiro': 400,
    'cereal_dinheiro': 200,
    'nut_dinheiro': 400,
    'milk_dinheiro': 500,
    'egg_dinheiro': 1000,
    'fish_dinheiro': 400,
    'honey_dinheiro': 5000,
    'fabric_dinheiro': 400,
    'cheese_dinheiro': 10000,
    'jam_dinheiro': 10000,
    'mushroom_dinheiro': 400,
    'spice_dinheiro': float(10000/15)
}

### Tipos de ganhos que podem ser gerados

 - Dinheiro -> Dinheiro: 'dinheiro_dinheiro'
 - Dinheiro -> Diamante: 'dinheiro_diamante'
 - Diamante -> Dinheiro: 'diamante_dinheiro'
 - Diamante -> Diamante: 'diamante_diamante'

## Crops

### Gerando realmente uma medida de valor

In [65]:
df_crops = dict_dfs['Crops'].copy()
CUSTO_BASE_CROPS = 10 #moeda = dinheiros

# Classificar cada item com seu tipo de ganho
df_crops.loc[(df_crops['moeda_custo'] == df_crops['moeda_ganho']) & (df_crops['moeda_custo'] == 'dinheiro'), 'tipo_ganho'] = 'dinheiro_dinheiro'
df_crops.loc[(df_crops['moeda_custo'] != df_crops['moeda_ganho']) & (df_crops['moeda_custo'] == 'dinheiro'), 'tipo_ganho'] = 'dinheiro_diamante'
df_crops.loc[(df_crops['moeda_custo'] != df_crops['moeda_ganho']) & (df_crops['moeda_custo'] == 'diamante'), 'tipo_ganho'] = 'diamante_dinheiro'
df_crops.loc[(df_crops['moeda_custo'] == df_crops['moeda_ganho']) & (df_crops['moeda_custo'] == 'diamante'), 'tipo_ganho'] = 'diamante_diamante'

df_crops['ganho_por_min'] = 0
df_crops.loc[((df_crops['tipo_ganho'] == 'dinheiro_dinheiro') | (df_crops['tipo_ganho'] == 'diamante_diamante')), 'ganho_por_min'] = (df_crops['ganho_base'] - (df_crops['custo'] + CUSTO_BASE_CROPS))/df_crops['tempo']
df_crops[['nome', 'custo', 'moeda_custo', 'ganho_base', 'xp', 'moeda_ganho', 'tempo', 'ganho_por_min', 'tipo_ganho']].sort_values('ganho_por_min', ascending=False)

,nome,custo,moeda_custo,ganho_base,xp,moeda_ganho,tempo,ganho_por_min,tipo_ganho
28,Jicama,2150.0,dinheiro,2220.0,6.0,dinheiro,10.0,6.000000,dinheiro_dinheiro
51,Black Goji Berry,2250.0,dinheiro,2400.0,10.0,dinheiro,25.0,5.600000,dinheiro_dinheiro
14,Romanesco,6000.0,dinheiro,6500.0,33.0,dinheiro,90.0,5.444444,dinheiro_dinheiro
42,Melon,1000.0,dinheiro,1060.0,5.0,dinheiro,10.0,5.000000,dinheiro_dinheiro
4,Purple Carrot,4200.0,dinheiro,4300.0,10.0,dinheiro,20.0,4.500000,dinheiro_dinheiro
...,...,...,...,...,...,...,...,...,...
96,Fern,0.0,0,0.0,0.0,dinheiro,0.0,0.000000,NaN
89,Flower Bouquets,0.0,dinheiro,0.0,0.0,dinheiro,180.0,-0.055556,dinheiro_dinheiro
82,Oregano,25.0,diamante,18.0,68.0,diamante,240.0,-0.070833,diamante_diamante
81,Habanero Pepper,3250.0,dinheiro,64.0,64.0,dinheiro,480.0,-6.658333,dinheiro_dinheiro


In [73]:
group_recurso = df_crops.groupby('tipo_recurso')
df_fruits = group_recurso.get_group('Fruits')
df_fruits[['nome', 'custo', 'moeda_custo', 'ganho_base', 'xp', 'moeda_ganho', 'tempo', 'ganho_por_min', 'tipo_ganho']].sort_values('ganho_por_min', ascending=False)

,nome,custo,moeda_custo,ganho_base,xp,moeda_ganho,tempo,ganho_por_min,tipo_ganho
42,Melon,1000.0,dinheiro,1060.0,5.0,dinheiro,10.0,5.000000,dinheiro_dinheiro
16,Crookneck Pumpkin,10000.0,dinheiro,12000.0,110.0,dinheiro,600.0,3.316667,dinheiro_dinheiro
47,Physalis,8000.0,dinheiro,11000.0,170.0,dinheiro,1440.0,2.076389,dinheiro_dinheiro
43,Pineapple,5500.0,dinheiro,7000.0,115.0,dinheiro,960.0,1.552083,dinheiro_dinheiro
44,Kiwifruit,1000.0,dinheiro,1600.0,86.0,dinheiro,1440.0,0.409722,dinheiro_dinheiro
46,Strawberry,800.0,dinheiro,1500.0,105.0,dinheiro,2160.0,0.319444,dinheiro_dinheiro
45,Watermelon,250.0,dinheiro,800.0,95.0,dinheiro,2880.0,0.187500,dinheiro_dinheiro
